# 8-Puzzle - Analysis of results

We're generating unique random puzzles, for which we're assigning 31 different weights per example ranging from 0.7 to 1.0 for the Manhattan distance heuristic and 0.0 to 0.3 for the Hamming distance heuristic.

The calculation time is mainly influenced by the depth of the solution path, as well as the mix of the weights of the heuristics. Typically the larger the Hamming distance becomes, the longer the calculation takes, therefore Hamming was capped at 0.3.

For the showcase we're taking 1000 unique puzzles, each having 31 weights, which results in 31.000 unique constellations.

All data set has the following columns:
'array', 'weights', 'visited_nodes', 'expanded_nodes','branching_factor', 'time_elapsed_astar', 'depth'


In [ ]:
import jsonpickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

with open("results_1000_2021-05-05.json", "r") as file:
    contents = file.read()
    unfrozen = jsonpickle.decode(contents)

df = pd.DataFrame(unfrozen)

In [ ]:
df.sort_values('depth', ascending=False)

# Nomenclature

(This may be interpreted in a few different ways, so I'll state it here)

- Visited Nodes. All nodes in the tree (visited in the sense as actually generated in the code)
- Expanded Nodes: All nodes which have children (non-leaf nodes)

## Puzzle-distribution by depth
Group results by depth (= how many steps were required to finish the puzzle), show the distribution of depths as chart

In [ ]:
group = df.groupby(['depth'])
x_pos = [i for i, _ in enumerate(group)]
fig, ax = plt.subplots(figsize=(15, 5))
plt.xticks(list(group.groups.keys()))
ax.bar(list(group.groups.keys()), list(group.size()))
plt.show()

# In all our previous tests it has shown that when using a random generation of puzzles,
# a seemingly gaussian distribution of puzzle-depths occurs


## Print constellation for all (average)

Shows the average expanded nodes based on the weight

### Average expanded & visited nodes

In [ ]:
weight_group = df.groupby(['weights'])

# Expanded nodes
x1 = np.array(list(weight_group.groups.keys()))[:, 0]
x2 = np.array(list(weight_group.groups.keys()))[:, 1]
y1 = weight_group['expanded_nodes'].mean()

# Visited nodes
y2 = weight_group['visited_nodes'].mean()

# Expanded nodes Manhattan
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[1].set_title("Expanded Nodes - Manhattan")
ax[1].plot(x1, y1, "b--", label="Manhattan weight")
ax[1].set_xlabel("Weight ratio")
ax[1].set_ylabel("Average expanded nodes")
ax[1].legend()

# Expanded nodes Hamming
ax[0].set_title("Expanded Nodes - Hamming")
ax[0].plot(x2, y1, "r", label="Hamming weight")
ax[0].set_xlabel("Weight ratio")
ax[0].set_ylabel("Average expanded nodes")
ax[0].legend()
plt.show()

# Visited nodes Manhattan
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[1].set_title("Visited Nodes - Manhattan")
ax[1].plot(x1, y2, "g--", label="Manhattan weight")
ax[1].set_xlabel("Weight ratio")
ax[1].set_ylabel("Average expanded nodes")
ax[1].legend()

# Visited nodes Hamming
ax[0].set_title("Visited Nodes - Hamming")
ax[0].plot(x2, y2, "m", label="Hamming weight")
ax[0].set_xlabel("Weight ratio")
ax[0].set_ylabel("Average expanded nodes")
ax[0].legend()
plt.show()

# We can see that expanded and visited nodes follow a very similar curve.

### Average Branching factor

In [ ]:
weight_group = df.groupby(['weights'])

# Branching factor
x1 = np.array(list(weight_group.groups.keys()))[:, 0]
x2 = np.array(list(weight_group.groups.keys()))[:, 1]
y1 = weight_group['branching_factor'].mean()
y2 = weight_group['branching_factor'].max()
y3 = weight_group['branching_factor'].min()


# Branching Factor Manhattan
fig, ax = plt.subplots(1,2, figsize=(15, 5))
ax[1].set_title("Branching Factor - Manhattan")
ax[1].plot(x1, y1, "r--", label="Manhattan - mean")
ax[1].plot(x1, y2, "g--", label="Manhattan - max")
ax[1].plot(x1, y3, "b--", label="Manhattan - min")
ax[1].set_xlabel("Weight ratio")
ax[1].set_ylabel("Branching Factor")
ax[1].legend()

# Branching factor Hamming
ax[0].set_title("Branching Factor - Hamming")
ax[0].plot(x2, y1, "r", label="Hamming - mean")
ax[0].plot(x2, y2, "g", label="Hamming - max")
ax[0].plot(x2, y3, "b", label="Hamming - min")
ax[0].set_xlabel("Weight ratio")
ax[0].set_ylabel("Branching Factor")
ax[0].legend()
plt.show()

############### MEAN IN DETAIL ###################

# Branching Factor Manhattan
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[1].set_title("Branching Factor - Manhattan - Mean")
ax[1].plot(x1, y1, "b--", label="Manhattan weight")
ax[1].set_xlabel("Weight ratio")
ax[1].set_ylabel("Average branching Factor")
ax[1].legend()

# Expanded nodes Hamming
ax[0].set_title("Branching Factor - Hamming - Mean")
ax[0].plot(x2, y1, "r", label="Hamming weight")
ax[0].set_xlabel("Weight ratio")
ax[0].set_ylabel("Average branching Factor")
ax[0].legend()
plt.show()


# Unexpected finding: The branching factor rises (very slightly) with rising Manhattan ratio


## Min Nodes

In [ ]:
min_value = group[['expanded_nodes']].min()
min_value.transpose()

## Max Nodes

In [ ]:
max_value = group[['expanded_nodes']].max()
max_value.transpose()

## Best weights based on the least nodes per depth

### Expanded

In [ ]:
min_nodes = df.loc[group['expanded_nodes'].idxmin()]
np.mean(min_nodes['weights'].tolist(), axis=0)

### Visited

In [ ]:
min_nodes = df.loc[group['visited_nodes'].idxmin()]
np.mean(min_nodes['weights'].tolist(), axis=0)

## Best weights based on the weights with the least nodes of each example

In [ ]:
group_by_example = df.groupby('array')

### Expanded

In [ ]:
min_expanded_nodes_per_example = df.loc[group_by_example['expanded_nodes'].idxmin()]
np.mean(min_expanded_nodes_per_example['weights'].tolist(), axis=0)

### Visited

In [ ]:
min_visited_nodes_per_example = df.loc[group_by_example['visited_nodes'].idxmin()]
np.mean(min_visited_nodes_per_example['weights'].tolist(), axis=0)

## Number of examples where Manhattan only ([1.0, 0.0]) is the best choice 

### Expanded / Visited

In [ ]:
manhattan_only_expanded = (min_expanded_nodes_per_example.loc[min_expanded_nodes_per_example['weights'] == (1.0, 0.0)]).shape[0]
print(f"Manhattan only is the best choice in {manhattan_only_expanded}/{min_expanded_nodes_per_example.shape[0]} examples for expanded nodes")

### Visited

In [ ]:
manhattan_only_visited = (min_visited_nodes_per_example.loc[min_visited_nodes_per_example['weights'] == (1.0, 0.0)]).shape[0]
print(f"Manhattan only is the best choice in {manhattan_only_visited}/{min_visited_nodes_per_example.shape[0]} examples for visited nodes")

## Best branching factor per example

In [ ]:
min_branching_factor_per_example = df.loc[group_by_example['branching_factor'].idxmin()]
np.mean(min_branching_factor_per_example['weights'].tolist(), axis=0)